<a href="https://colab.research.google.com/github/ntngoctram98/Compress-WSQ/blob/master/NMMH_TH8_1611297.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bài 8: Keras, Regression

Trong bài thực hành này, chúng ta sẽ 
- Làm quen với Tensorflow thông qua các bài toán Regression


## 1. Dự đoán giá nhà (Lab 01)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/dinhvietcuong1996/Lab-MachineLearningCourse/master/Lab08/House_Price.csv")   
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1)

## dùng width và length để đoán price
X_train = df_train[['width', 'length']].values    
X_test = df_test[['width', 'length']].values

X_means = np.mean(X_train, axis=0)
X_stds = np.std(X_train, axis=0)
X_train = (X_train - X_means)/X_stds
X_test = (X_test - X_means)/X_stds

y_train = df_train['price'].values
y_test = df_test['price'].values

y_means = np.mean(y_train, axis=0)
y_stds = np.std(y_train, axis=0)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(2,))    ## tạo một layer Input, mỗi sample có input là 1 vector độ dài 2
dense_1 = Dense(units=10, activation='relu')(inputs)   ## tạo một layer gồm 10 units, hàm kích hoạt relu
outputs = Dense(units=1, activation=None)(dense_1)   ## tạo layer output gồm 1 units, tượng trưng cho giá nhà, vì giá nhà thực nên không để hàm kích hoạt
model = Model(inputs=inputs, outputs=outputs)    ## tạo model với input và output layer đã xây dựng

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1000.0),
             loss='mean_squared_error')      ## khai báo optimizer và loss cho model
model.summary()           

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=80)

## 2. Bài tập: COVID-19 Retweet Prediction

Link: https://www.cikm2020.org/covid-19-retweet-prediction-challenge/

Dự đoán số lượng retweet của một tweet dựa vào các metadata như số #friends, #followers, ... của người đăng, thời gian, hashtags của tweet.

Download: https://drive.google.com/drive/folders/1eZwesVW9RcW_0ZJt0ztGyhTlISx3USn5?usp=sharing

In [ ]:
df_train = pd.read_csv("df_train.csv")
df_train.head()

## Bài tập
Sử dụng một vài hoặt tất cả các cột trong số các cột timestamp, #followers, #friends, #favorites, hashtags, sentiment_1, sentiment_2 để dự đoán cột #retweet:
- Dùng model machine learning bất kì (nên dùng neural network)
- Đạt Mean Squared Logarithmic Error (https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredLogarithmicError) trên tập test ít nhất là 0.25 mới được đầy đủ điểm

## Nộp bài
- Code và chạy kết quả lưu vào file notebook NMMH_TH8_MSSV.ipynb (notebook phải có kết quả chạy nếu ko xem như chưa làm)
- Nén thành file NMMH_TH8_MSSV.rar (.zip) và nộp về: dinhvietcuong1996@gmail.com
- Deadline: 23g59 thứ 3 ngày 28/07/2020. Nộp trễ bị chia đôi số điểm.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(df_train, test_size=0.3)

In [ ]:
import statsmodels.api as sm

def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

In [ ]:
column = []
for i in df_train.columns:
  if (df_train.dtypes[i] != 'object' and i != '#retweet'):
    column.append(i)
column

In [ ]:
forward_selection(df_train[column], df_train['#retweet'])

In [ ]:
df_test = pd.read_csv("df_test.csv")
X_test = df_test[column].values
X_test = (X_test - X_means)/X_stds
y_test = df_test['#retweet'].values

In [ ]:
X_train = df_train[column].values    
X_valid = df_valid[column].values

X_means = np.mean(X_train, axis=0)
X_stds = np.std(X_train, axis=0)
X_train = (X_train - X_means)/X_stds
X_valid = (X_valid - X_means)/X_stds

y_train = df_train['#retweet'].values
y_valid = df_test['#retweet'].values

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(5,))    ## tạo một layer Input, mỗi sample có input là 1 vector độ dài 5
dense_1 = Dense(units=10, activation='relu')(inputs)   ## tạo một layer gồm 10 units, hàm kích hoạt relu
outputs = Dense(units=1, activation=None)(dense_1)  
model = Model(inputs=inputs, outputs=outputs)    ## tạo model với input và output layer đã xây dựng

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')      ## khai báo optimizer và loss cho model
model.summary()

In [ ]:
df_test = pd.read_csv("df_test.csv")
X_test = df_test[column].values
X_test = (X_test - X_means)/X_stds
y_test = df_test['#retweet'].values

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.round(y_pred)
y_pred = y_pred.astype(int)
y_pred = np.squeeze(y_pred)

In [ ]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(y_test, y_pred)